<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/ModData/Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Base T5**.

In [ ]:
colab = False

In [ ]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    !git checkout seq2seq
    %pip install -r requirements_colab.txt
    

# 1.0 Preparation


Set parameters for the experiments.

In [ ]:
sizes = {
    'n_train' : 10000,
    'n_val' : 1000,
    'n_test' : 1000
}

NUM_EPOCHS = 1

# Whether to use the mnli prompt on which the model is pretrained or not
USE_MNLI_PROMPT = False

## 1.1 Loading Tokenizer

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-base", truncation=True, padding=True)

## 1.2 Loading and Tokenizing Dataset

In [ ]:
from datasets import load_dataset
from src.preprocess import prepare_dataset
from functools import partial
from src.utils import tokenize_function

In [ ]:
dataset = load_dataset("esnli", download_mode="force_redownload")

In [ ]:
tokenize_mapping= partial(tokenize_function, tokenizer=tokenizer, use_mnli_format = USE_MNLI_PROMPT)

In [ ]:
train_tok, valid_tok, test_tok = prepare_dataset(dataset, tokenize_mapping=tokenize_mapping, sizes = sizes)

## 1.3 Loading SBERT for evaluating sentence similarity

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# 2.0 Tasks

In [ ]:
import torch
from functools import partial
import evaluate
from src.utils import compute_metrics, eval_pred_transform_accuracy
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration, DataCollatorForSeq2Seq


In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device

In [ ]:
transform_accuracy = partial(eval_pred_transform_accuracy, tokenizer = tokenizer)
compute_accuracy = partial(compute_metrics, pred_transforms=transform_accuracy, metrics = evaluate.load('accuracy'))

## 2.1 Task 1: Zero-shot evaluation

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="task1",
    predict_with_generate=True,
    per_device_eval_batch_size=16,
    generation_max_length=32,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=valid_tok,
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.evaluate(test_tok)

## 2.2 Task 2: Fine tuning without explanations

In [ ]:
NUM_EPOCHS = 1

In [ ]:
model_ft = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator_ft = DataCollatorForSeq2Seq(tokenizer, model=model_ft)

In [ ]:
training_args_ft = Seq2SeqTrainingArguments(
    save_strategy="no",
    output_dir="task2",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=32,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft = Seq2SeqTrainer(
    model=model_ft,
    args=training_args_ft,
    train_dataset=train_tok,
    eval_dataset=valid_tok,
    compute_metrics=compute_accuracy,
    data_collator=data_collator_ft,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft.train()

In [ ]:
trainer_ft.evaluate(test_tok)

## 2.3 Task 3: Fine Tuning with Explanations

We need to give as labels the label and the explanation tokenized.

### Preparing the dataset with labelled explanations

In [ ]:
from src.utils import tokenize_function_ex

In [ ]:
dataset_explanations = load_dataset("esnli", download_mode="force_redownload")

In [ ]:
tokenize_mapping_ex = partial(tokenize_function_ex, tokenizer=tokenizer, use_mnli_format = USE_MNLI_PROMPT)

In [ ]:
train_tok_ex, valid_tok_ex, test_tok_ex = prepare_dataset(dataset=dataset_explanations, tokenize_mapping=tokenize_mapping_ex, sizes=sizes)

In [ ]:
train_tok_ex.features

### Defining the metrics: accuracy / similarity of explanations

In [ ]:
from src.utils import eval_pred_transform_sbert
from sbert_metric import SbertMetric

In [ ]:
transform_accuracy_ex = partial(eval_pred_transform_accuracy, tokenizer = tokenizer, remove_explanations_from_label = True)
accuracy = evaluate.load('accuracy')

In [ ]:
trasnform_sbert = partial(eval_pred_transform_sbert, tokenizer = tokenizer)
sbert_similarity = SbertMetric(sbert)

In [ ]:
transforms = [transform_accuracy_ex, trasnform_sbert]
metrics = [accuracy, sbert_similarity]

compute_metrics_ex = partial(compute_metrics, pred_transforms=transforms, metrics=metrics)

### Fine Tuning

In [ ]:
NUM_EPOCHS = 1

In [ ]:
model_ft_ex = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator_ft_ex = DataCollatorForSeq2Seq(tokenizer, model=model_ft_ex)

In [ ]:
training_args_ft_ex = Seq2SeqTrainingArguments(
    save_strategy="no",
    output_dir="task3",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=128,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft_ex = Seq2SeqTrainer(
    model=model_ft_ex,
    args=training_args_ft_ex,
    train_dataset=train_tok_ex,
    eval_dataset=valid_tok_ex,
    compute_metrics=compute_metrics_ex,
    data_collator=data_collator_ft_ex,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft_ex.train()

In [ ]:
trainer_ft_ex.evaluate(test_tok_ex)

## 2.4 Task 4: Fine Tuning with Shuffled Explanations

### Preparing the dataset with *wrong* labelled explanations

In [ ]:
dataset_shex = load_dataset("esnli", download_mode="force_redownload")

In [ ]:
from src.preprocess import save_explanations, save_shuffled_explanations, retrieve_explanations

In [ ]:
dirs = save_explanations(dataset_shex)

In [ ]:
dirs_shuffled = save_shuffled_explanations(dirs)

In [ ]:
shuffled_explanations = retrieve_explanations(dirs_shuffled)

In [ ]:
from src.utils import tokenize_function_ex

tokenize_mapping_train = partial(tokenize_function_ex, tokenizer=tokenizer, explanations = shuffled_explanations['train'], use_mnli_format = USE_MNLI_PROMPT)
tokenize_mapping_val = partial(tokenize_function_ex, tokenizer=tokenizer, explanations = shuffled_explanations['validation'], use_mnli_format = USE_MNLI_PROMPT)
tokenize_mapping_test = partial(tokenize_function_ex, tokenizer=tokenizer, explanations = shuffled_explanations['test'], use_mnli_format = USE_MNLI_PROMPT)

tokenize_mappings = (tokenize_mapping_train, tokenize_mapping_val, tokenize_mapping_test)

In [ ]:
train_tok_shex, valid_tok_shex, test_tok_shex = prepare_dataset(dataset, tokenize_mapping=tokenize_mappings, sizes=sizes)

In [ ]:
train_tok_shex = train_tok_shex.remove_columns(["explanation_1", "explanation_2", "explanation_3"])
valid_tok_shex = valid_tok_shex.remove_columns(["explanation_1", "explanation_2", "explanation_3"])
test_tok_shex = test_tok_shex.remove_columns(["explanation_1", "explanation_2", "explanation_3"])

### Fine Tuning

In [ ]:
NUM_EPOCHS = 1

In [ ]:
model_ft_shex = T5ForConditionalGeneration.from_pretrained("t5-base")
data_collator_ft_shex = DataCollatorForSeq2Seq(tokenizer, model=model_ft_ex)

In [ ]:
training_args_ft_shex = Seq2SeqTrainingArguments(
    save_strategy="no",
    output_dir="task4",
    evaluation_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    generation_max_length=128,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer_ft_shex = Seq2SeqTrainer(
    model=model_ft_shex,
    args=training_args_ft_shex,
    train_dataset=train_tok_shex,
    eval_dataset=valid_tok_shex,
    compute_metrics=compute_metrics_ex,
    data_collator=data_collator_ft_shex,
    tokenizer=tokenizer,
)

In [ ]:
trainer_ft_shex.train()

In [ ]:
trainer_ft_shex.evaluate(test_tok_shex)

## 2.5 Task 5: Profiling-UD

### Read the results of the automatic annotation stage performed over explanations with Profilind-UD.

1. **Token ID**: The token's position in the sentence.
2. **Token**: The actual token text.
3. **Lemma**: The lemma or base form of the token.
4. Universal part-of-speech tag.
5. Language-specific part-of-speech tag (optional).
6. Miscellaneous (misc) field, which can contain additional annotations.
7. Head: The ID of the token's syntactic head.
8. Dependency relation: The type of syntactic relation between the token and its head.
9. Secondary dependencies or additional annotations.

In [ ]:
import pandas as pd 
# Define the path to your CoNLL-U file
conll_file_path = "ex_files/explanations_train.conllu"

# Define column names for the CoNLL-U file
column_names = [
    "ID",
    "TOKEN",
    "LEMMA",
    "UPOS",
    "XPOS",
    "FEATS",
    "HEAD",
    "DEPREL",
    "DEPS",
    "MISC"
]

# Read the CoNLL-U file into a DataFrame
df = pd.read_csv(conll_file_path, delimiter='\t', comment='#', header=None, names=column_names)

# Reset the index to create a numeric index
df.reset_index(drop=True, inplace=True)

# Display the DataFrame
df[:15]

In [ ]:
df['SAMPLE'] = None

sample = 0
for index, row in df.iterrows():
    if(row["ID"]==1):
        sample = sample+1
    df.at[index, "SAMPLE"] = sample

### Prepare the dataset with modified explanations

### Prepare the dataset with modified explanations

In [ ]:
# Define the input and output file paths
output_file = "ex_files/modified_explanations_1.txt"

# Write the shuffled lines to the output file
with open(output_file, "w") as f:
    for i in range(N_TRAIN):
        df_i = df.loc[df["SAMPLE"]==i]
        modified_exp = ' '.join(df["LEMMA"].values)
        f.writelines(modified_exp)

In [ ]:
with open("ex_files/modified_explanations_1.txt", "r") as f:
    explanations_m1 = f.readlines()

In [ ]:
# Define the input and output file paths
output_file = "ex_files/modified_explanations_1.txt"

# Write the shuffled lines to the output file
with open(output_file, "w") as f:
    for i in range(N_TRAIN):
        df_i = df.loc[df["SAMPLE"]==i]
        modified_exp = ' '.join(df["LEMMA"].values)
        f.writelines(modified_exp)

In [ ]:
with open("ex_files/modified_explanations_1.txt", "r") as f:
    explanations_m1 = f.readlines()